In [1]:
import os
import sys
import random
import cv2
import numpy as np
import tensorflow as tf

In [2]:
sys.path.append("../facenet/src")
import facenet
import align.detect_face

In [3]:
with tf.Graph().as_default():
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
# Face Detection constants.
MIN_FACE_SIZE = 20                     # minimum size of the face for the MTCNN
DETECT_THRESHOLDS = [ 0.6, 0.7, 0.7 ]  # threshold values for the three stages of the MTCNN
SCALE_FACTOR = 0.709                   # MTCNN scale factor

# Face Embedding constants.
INPUT_IMAGE_SIZE = 224

# This function normalizes the image before generating the embedding.
def run_facenet(image):
    image_data = np.around(image/255.0, decimals=12)
    image_data = np.expand_dims(image_data, axis=0)
    return enet(image_data)

def load_image(image_path):
    errorMessage = ''
    
    img = cv2.imread(image_path)
    
    if (img is None):
        errorMessage = '{}: failed to load'.format(image_path)
        return None, errorMessage
    
    height, width, channels = img.shape 
    
    if (channels < 3):
        errorMessage = '{}: less than three dimensions'.format(image_path)
        return None, errorMessage
    
    # Remove Alpha
    img = img[:,:,0:3]
    
    # convert to RGB.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        
    return img, errorMessage

def import_dataset(input_dir, output_dir, log_filename):   
    output_dir = os.path.expanduser(output_dir)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    dataset = facenet.get_dataset(input_dir)
    
    log_file = open(os.path.join(output_dir, log_filename), "w")
    
    images_total = 0
    successfully_aligned = 0
    
    for cls in dataset:
        output_class_dir = os.path.join(output_dir, cls.name)
        
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)
        
        for image_path in cls.image_paths:
            images_total += 1
            filename = os.path.splitext(os.path.split(image_path)[1])[0]
            output_filename = os.path.join(output_class_dir, filename+'.png')
            
            print(image_path)
            
            if not os.path.exists(output_filename):
                img, errorMessage = load_image(image_path)

                if (img is None):
                    print(errorMessage)
                    log_file.write(errorMessage)
                    continue

                bounding_boxes, _ = align.detect_face.detect_face(img, MIN_FACE_SIZE, pnet, rnet, onet, DETECT_THRESHOLDS, SCALE_FACTOR)
                
                faces = bounding_boxes.shape[0]
                
                if faces>0:
                    det = bounding_boxes[:,0:4]
                    det_arr = []
                    img_size = np.asarray(img.shape)[0:2]
                    if faces>1:
                        #bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                        #img_center = img_size / 2
                        #offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                        #offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                        #index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                        #det_arr.append(det[index,:])
                        errorMessage = '{}: {} faces'.format(image_path, faces)
                        print(errorMessage)
                        log_file.write(errorMessage)
                    else:
                        det_arr.append(np.squeeze(det))

                    for i, det in enumerate(det_arr):
                        det = np.squeeze(det)
                        bb = np.zeros(4, dtype=np.int32)
                        bb[0] = np.maximum(det[0], 0)
                        bb[1] = np.maximum(det[1], 0)
                        bb[2] = np.minimum(det[2], img_size[1])
                        bb[3] = np.minimum(det[3], img_size[0])
                        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                        scaled = cv2.resize(cropped, (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE), interpolation=cv2.INTER_LINEAR) 
                        successfully_aligned += 1
                        filename_base, file_extension = os.path.splitext(output_filename)                                
                        output_filename_n = "{}{}".format(filename_base, file_extension)
                        scaled = cv2.cvtColor(scaled, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(output_filename_n,scaled)
                else:
                    errorMessage = '{}: no faces'.format(image_path)
                    print(errorMessage)
                    log_file.write(errorMessage)
    
    print('Total number of images: %d' % images_total)
    print('Number of successfully aligned images: %d' % successfully_aligned)
    log_file.close()

In [5]:
input_dir = "data/facescrub"
output_dir = "data/facescrub-output-224"
import_dataset(input_dir, output_dir, "logfile.txt")

data/facescrub/Aaron_Eckhart/008a9b44d28e03ec64f26e580bb778e2febd7908.jpg
data/facescrub/Aaron_Eckhart/02454bf4214b50ea5df5bc01498f474471356363.jpg
data/facescrub/Aaron_Eckhart/02695c01048091b63a3151ae10166e30ee25b711.jpg
data/facescrub/Aaron_Eckhart/07659db54fa8b7a823d8acec4252ab859e4157d7.jpg
data/facescrub/Aaron_Eckhart/088ed6e1c13c6bb3e1229e9353160c255e357581.jpg
data/facescrub/Aaron_Eckhart/0ae7f79b53f769ada53d9062e040883cafa35746.jpg
data/facescrub/Aaron_Eckhart/0bb7972fe6e2d8b843bf7cde62ad410c1de0bd69.jpg
data/facescrub/Aaron_Eckhart/0de49b52c4db40e876d2fad7130add059f886f62.jpg
data/facescrub/Aaron_Eckhart/0e78721478c675c9e2a0eddecc7ed3bd08360b5f.jpg
data/facescrub/Aaron_Eckhart/0f7d31e3037816d69eefa8e2f0c04ce73b93069f.jpg
data/facescrub/Aaron_Eckhart/0f7d31e3037816d69eefa8e2f0c04ce73b93069f.jpg: 4 faces
data/facescrub/Aaron_Eckhart/14816cb0483902ee94f6282303b06d941c287077.jpg
data/facescrub/Aaron_Eckhart/15b172215780acd9fe67a413d279e9fd23674c10.jpg
data/facescrub/Aaron_Eckhart/

data/facescrub/Aaron_Eckhart/b837db1058c0e2731732acfb3618407237a3a52e.jpg
data/facescrub/Aaron_Eckhart/b849926627ad625847313b9969fbd73f1f656e3b.jpg
data/facescrub/Aaron_Eckhart/b87f06eded3d7300e486f8eb4556923d1d952869.jpg
data/facescrub/Aaron_Eckhart/b8e5a10140257f7a2d621e52f1547a7ab2f80c20.jpg
data/facescrub/Aaron_Eckhart/b8e5a10140257f7a2d621e52f1547a7ab2f80c20.jpg: 2 faces
data/facescrub/Aaron_Eckhart/b91d929d79aec9cf2f2a6ceb8a7bbcec433dba5a.jpg
data/facescrub/Aaron_Eckhart/bb8d56c2adbac049f58449bd7432b07b9637616d.jpg
data/facescrub/Aaron_Eckhart/bd3206150dd7db447c56584ac2965bb74d398e93.jpg
data/facescrub/Aaron_Eckhart/beb4e705d7c9fb997d7ed8e9a75a9069d6feb244.jpg
data/facescrub/Aaron_Eckhart/c158eabb6b7b94902281c6a1c27f1063879708b7.jpg
data/facescrub/Aaron_Eckhart/c43dd82303ead5a1df292c09c0cc205d14f4d63d.jpg
data/facescrub/Aaron_Eckhart/c4671ca46599d374b96954492042b48720f24fcc.jpg
data/facescrub/Aaron_Eckhart/c67f5711bd58668dcada9ac60d8e04a77067fc8c.jpg
data/facescrub/Aaron_Eckhart/

data/facescrub/Adam_Brody/75301be09451d0b9ad0f77ce57eef4dc8824ff8c.jpg
data/facescrub/Adam_Brody/7536b666ea07edc7be759a4b3e979f913551bcbb.jpg
data/facescrub/Adam_Brody/77f6165e018789a4039b17f1f02020479d9d54f9.jpg
data/facescrub/Adam_Brody/79bf9ad6918cdbd850398e36eff8b03444c8f422.jpg
data/facescrub/Adam_Brody/79bf9ad6918cdbd850398e36eff8b03444c8f422.jpg: 4 faces
data/facescrub/Adam_Brody/7a8eec252ba40c9f1f8f083793885d959ed91faa.jpg
data/facescrub/Adam_Brody/7b71d848ea8b415449d519bffdebab2da90c4602.jpg
data/facescrub/Adam_Brody/7b71d848ea8b415449d519bffdebab2da90c4602.jpg: 4 faces
data/facescrub/Adam_Brody/8080cf47a2c1c5788ac2f2c833e4a57fba25c80c.jpg
data/facescrub/Adam_Brody/8080cf47a2c1c5788ac2f2c833e4a57fba25c80c.jpg: 2 faces
data/facescrub/Adam_Brody/8711988d93d990c0b2194deec2c868400026706d.jpg
data/facescrub/Adam_Brody/8775ad7d2322c59fa2c10ca0605086f0093d283c.jpg
data/facescrub/Adam_Brody/8ab65177ed6561ffc668164e15b5d3b075d45c06.jpg
data/facescrub/Adam_Brody/8ab65177ed6561ffc668164e

data/facescrub/Adam_McKay/40a24f3e83f03b94ffa3f0a73937e58457d41667.jpg: 4 faces
data/facescrub/Adam_McKay/428efe4e36e7454664a435fb23b43a0dc6affc65.jpg
data/facescrub/Adam_McKay/462efd7215b13f65d8f366e508e639c37b8eeb53.jpg
data/facescrub/Adam_McKay/4a648aec537e930fc4c7237b983905611ef1676f.jpg
data/facescrub/Adam_McKay/4a648aec537e930fc4c7237b983905611ef1676f.jpg: 3 faces
data/facescrub/Adam_McKay/508de984ce6c3d3cda50cb3c92abe154a1257238.jpg
data/facescrub/Adam_McKay/514f90b21efa53c59d262545d9afcc71d1f3ea95.jpg
data/facescrub/Adam_McKay/58de03667d8a86894b1a7cb356f8ec2b7b347d59.jpg
data/facescrub/Adam_McKay/58de03667d8a86894b1a7cb356f8ec2b7b347d59.jpg: 2 faces
data/facescrub/Adam_McKay/5b7cfdb304722d3bfd3d85174addcbf79cd80fa0.jpg
data/facescrub/Adam_McKay/5beab2d2a0d7ebbe0b783a083c7b600ba97398fe.jpg
data/facescrub/Adam_McKay/5beab2d2a0d7ebbe0b783a083c7b600ba97398fe.jpg: 2 faces
data/facescrub/Adam_McKay/5e09c0ae486117f4a1301f1edfac9de079b604e6.jpg
data/facescrub/Adam_McKay/5e09c0ae486117f

data/facescrub/Adam_Sandler/46778ad36920a30d8dfa596e5c8d6d9cb03d5bd1.jpg: 2 faces
data/facescrub/Adam_Sandler/46b59c7bf3cab3f44b07839ce2c2af76b57dd1ed.jpg
data/facescrub/Adam_Sandler/47de99520573cd80cb810aab8993aa6d9b5e049a.jpg
data/facescrub/Adam_Sandler/492375e1ad4c2035fd182be92403f890887e7fe0.jpg
data/facescrub/Adam_Sandler/4a360f8994745265b64908e1936640be35df70fd.jpg
data/facescrub/Adam_Sandler/4a360f8994745265b64908e1936640be35df70fd.jpg: 5 faces
data/facescrub/Adam_Sandler/4d6ac3864ba16682f0898e324d3957ab21c710fd.jpg
data/facescrub/Adam_Sandler/4ecd3220dafeb1c3ba7aad5a51195f69da165fc2.jpg
data/facescrub/Adam_Sandler/4ee3a1ed7bacac18a452fca363342ca01aec644e.jpg
data/facescrub/Adam_Sandler/4efb190aef7f171fb4b2a35ba0663d9d1f7b3b62.jpg
data/facescrub/Adam_Sandler/4f380485a3329386b5a46c16319aefcf1960dfff.jpg
data/facescrub/Adam_Sandler/4f380485a3329386b5a46c16319aefcf1960dfff.jpg: 2 faces
data/facescrub/Adam_Sandler/4f7c866f33e297fbf26e6d7e855bb17fea479ccc.jpg
data/facescrub/Adam_Sand

...

data/facescrub/Annie_Ilonzeh/a6fb25800ece1430bcf4f3f89b9a2528e7fe4d25.jpg: 4 faces
data/facescrub/Annie_Ilonzeh/a7482c86bd69620ce75fae357ae9cba9ac9de25c.jpg
data/facescrub/Annie_Ilonzeh/a7e2a4c8dc92748b74a2f2abf537ea52b3236fb3.jpg
data/facescrub/Annie_Ilonzeh/a8ae3211c8ff248201fa3ecec37d1161dc950387.jpg
data/facescrub/Annie_Ilonzeh/aaba436b22f9bdd5752775ddd24e37327f7af073.jpg
data/facescrub/Annie_Ilonzeh/ac960840e54c2882af98e31db5f3317c90cf5bc8.jpg
data/facescrub/Annie_Ilonzeh/b2f20177cae201b1b942e2104b4be5f200b9d225.jpg
data/facescrub/Annie_Ilonzeh/bbd8def059d32fe6fd9e02435c97040c2dc4a240.jpg
data/facescrub/Annie_Ilonzeh/bbff989beb062763c5ae38fe94796c43a1f04541.jpg
data/facescrub/Annie_Ilonzeh/bea0f5d23ebe98b2d402365734f8757d1c1ed42d.jpg
data/facescrub/Annie_Ilonzeh/c0cfc8181626ca840770dec5daf7c72483c1cf29.jpg
data/facescrub/Annie_Ilonzeh/c2e437845ceebbd6300dfa834a8aa3201da1a767.jpg
data/facescrub/Annie_Ilonzeh/c2e437845ceebbd6300dfa834a8aa3201da1a767.jpg: 2 faces
data/facescrub/Annie

data/facescrub/Anthony_Hopkins/47f5e4a123be1d533641aad3a0281ec699ee2b0e.jpg
data/facescrub/Anthony_Hopkins/4f18ede30d15c9e9ceb47d99c0d36422c7cc7347.jpg
data/facescrub/Anthony_Hopkins/51d1cfbc8af84aac1f4b6f3f97343ab4f361bf73.jpg
data/facescrub/Anthony_Hopkins/51f8070ed5042101dc43cc60e64d3a61f51c2b90.jpg
data/facescrub/Anthony_Hopkins/53c8f3084511638f755cc29e131605dad175cfcd.jpg
data/facescrub/Anthony_Hopkins/5588eaa17037fde8101281225d0b5a4ac0205cff.jpg
data/facescrub/Anthony_Hopkins/55ea1aa7503efa9f83ba08c082200aa86320cfd0.jpg
data/facescrub/Anthony_Hopkins/5dbf5f5615f13451a1742cf098428035d5dba681.jpg
data/facescrub/Anthony_Hopkins/60612e48084e86f72d2013ce37814271a9e2dfa7.jpg
data/facescrub/Anthony_Hopkins/62b10f5ff85c00d4077ab15c2d20f17d243fed5d.jpg
data/facescrub/Anthony_Hopkins/633352bb618933e708af594b3b369cd9227a5d87.jpg
data/facescrub/Anthony_Hopkins/638af04c9c07297586a78e0d98ff5de6ba839d40.jpg
data/facescrub/Anthony_Hopkins/63dc8fac26c952c56cb5a2f51c49e4718b49cc5f.jpg
data/facescr

data/facescrub/Anthony_Hopkins/ecc7b2922c9bb4f334684aced4412bf337677dee.jpg
data/facescrub/Anthony_Hopkins/f5268f4109bb881c107079cc416fb724516db2ae.jpg
data/facescrub/Anthony_Hopkins/f6da82d998b65a20c70f00d1b529f9732c63c5ef.jpg
data/facescrub/Anthony_Hopkins/f71f048376bf9e30c0239ca193201756324c6b17.jpg
data/facescrub/Anthony_Hopkins/f758748f5893ada23243ed40077197b84974f1d1.jpg
data/facescrub/Anthony_Hopkins/f8bb8decfdf2a82f4de85b05146e8776279698b3.jpg
data/facescrub/Anthony_Hopkins/f9f7907316dedb0b604a245d2fe441661532ddad.jpg
data/facescrub/Anthony_Hopkins/face
data/facescrub/Anthony_Hopkins/face: failed to load
data/facescrub/Anthony_Hopkins/fb12658f2791ab6b02fa5f0537a7a956bb2bafc2.jpg
data/facescrub/Anthony_Hopkins/fb5095364f662a48ed34e1883477e77102081f4e.jpg
data/facescrub/Anthony_Stewart_Head/013267a6b2925da445e675c4df12037973ab5e6c.jpg
data/facescrub/Anthony_Stewart_Head/01f57593f4263d8ef1a8e61daa4e3074680d9fc9.jpg
data/facescrub/Anthony_Stewart_Head/046385930dc96152df7c7deffae11f

data/facescrub/Anthony_Stewart_Head/9bdd6cd5adbfb7d32210efbc1d578123df4873bd.jpg: 2 faces
data/facescrub/Anthony_Stewart_Head/9d18c3ac6be8c575ef1fac78a8d28a6dee60e7ff.jpg
data/facescrub/Anthony_Stewart_Head/9d18c3ac6be8c575ef1fac78a8d28a6dee60e7ff.jpg: 3 faces
data/facescrub/Anthony_Stewart_Head/_bboxes.txt
data/facescrub/Anthony_Stewart_Head/_bboxes.txt: failed to load
data/facescrub/Anthony_Stewart_Head/a009edf687c5d731db079b6e3a0a70f198ab33b5.jpg
data/facescrub/Anthony_Stewart_Head/a0322a26cd6a275c2a4771a24de89ff69937162c.jpg
data/facescrub/Anthony_Stewart_Head/a2ba4c1eb23fefbadf775f25b7dd2fda2319f7c1.jpg
data/facescrub/Anthony_Stewart_Head/a4412a33e7c19e124c5ca134520f517ce94ddc24.jpg
data/facescrub/Anthony_Stewart_Head/a5d07557eeccfa32d842240be20c101b1ec5b918.jpg
data/facescrub/Anthony_Stewart_Head/a6cd59178a42c0d89a38fc96029012e694f08298.jpg
data/facescrub/Anthony_Stewart_Head/a7365541286d069ab118b64b8e80ab453fb2d17c.jpg
data/facescrub/Anthony_Stewart_Head/a7f64a232b7344016e65b9bb

KeyboardInterrupt: 